In [ ]:
# IMPORTING NECESSARY MODULES

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image

In [ ]:
# STORING DATA AND LABELS
# dataset link is provided in README.md
import os
 
data = []
labels = []
 
classes = 43   #GTSRB has 43 classes
 
cwd = os.getcwd()
 
#print(cwd)


In [ ]:
# PREPROCESSING THE IMAGE

for img in range(classes):
    path = os.path.join(cwd, 'Train', str(img))
    #print(path)
    images = os.listdir(path)
    #print(images)
    for i in images:
        try:
            image = Image.open(path + '/' + i)
            image = image.resize((32,32))
            image = np.array(image)
            data.append(image)
            labels.append(img)
        except Exception as e:
            print(e)
    

In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
# SAVE THE LABELS AND DATA FOR FUTURE USES
# os.mkdir('trained_data')

np.save('trained_data/data', data)
np.save('trained_data/labels', labels)
# np.save()

In [ ]:
# LOAD THE NP DATA AND LABELS

data = np.load('trained_data/data.npy')
labels = np.load('trained_data/labels.npy')

In [ ]:
data.shape

In [ ]:
labels.shape

In [ ]:
# SPLIT THE DATA

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


In [ ]:
# LABELS TO ONE-HOT ENCODING

'''This code is converting the labels in y_train and y_test to one-hot encoded vectors with 43 classes. '''

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)



In [ ]:
y_train

# CNN Model Architecture

This model is a Convolutional Neural Network (CNN) designed for image classification tasks. Below is a summary of the model architecture:

## Layers

1. **Conv2D Layer**
   - **Filters:** 32
   - **Kernel Size:** (5, 5)
   - **Activation Function:** ReLU
   - **Input Shape:** Same as the input image shape, excluding the batch size (e.g., `(32, 32, 3)` for an RGB image with 32x32 pixels).

2. **Conv2D Layer**
   - **Filters:** 32
   - **Kernel Size:** (5, 5)
   - **Activation Function:** ReLU

3. **MaxPooling2D Layer**
   - **Pool Size:** (2, 2)

4. **Dropout Layer**
   - **Rate:** 0.25

5. **Conv2D Layer**
   - **Filters:** 64
   - **Kernel Size:** (3, 3)
   - **Activation Function:** ReLU

6. **Conv2D Layer**
   - **Filters:** 64
   - **Kernel Size:** (3, 3)
   - **Activation Function:** ReLU

7. **MaxPooling2D Layer**
   - **Pool Size:** (2, 2)

8. **Dropout Layer**
   - **Rate:** 0.25

9. **Flatten Layer**
   - Converts the 2D matrix into a 1D vector for the dense layer.

10. **Dense Layer**
    - **Units:** 256
    - **Activation Function:** ReLU

11. **Dropout Layer**
    - **Rate:** 0.5

12. **Dense Layer**
    - **Units:** 43 (number of classes)
    - **Activation Function:** Softmax (used for multi-class classification)


In [ ]:
# USING TENSORBOARD
from tensorflow.keras.callbacks import TensorBoard
import datetime


In [ ]:
# BUILDING THE MODEL

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(5,5), activation='relu',input_shape=X_train.shape[1:]),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(5,5), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=((2,2))),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(43, activation="softmax")                           
])

In [ ]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=15, validation_data=(X_test,y_test), callbacks=[tensorboard_callback])


In [ ]:
# tensorboard --logdir=logs/fit

'''Use this in the terminal to analyse your model'''

In [ ]:
# WITOUT TENSORBOARD

plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()


In [ ]:
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# EVALUATING OUR MODEL

def test_model(test_csv):
    y_test = pd.read_csv(test_csv)
    label = y_test['ClassId'].values
    imgs = y_test['Path'].values
    data = []
    
    for img in imgs:
        image = Image.open(img)
        image = image.resize((32,32))
        data.append(np.array(image))
    X_test = np.array(data)
    return X_test, label

In [ ]:
X_test, label = test_model('Test.csv')

In [ ]:
X_test

In [ ]:
y_pred_prob = model.predict(X_test)

# Find the class with the highest probability
y_pred = np.argmax(y_pred_prob, axis=-1)

In [ ]:
y_pred

In [ ]:
model.save("GTSRB.h5")

In [ ]:
# CHECK ACCURACY

from sklearn.metrics import accuracy_score
from keras.models import load_model

In [ ]:
model = load_model("GTSRB.h5")

In [ ]:
# CLASSES USED IN THIS DATASET
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

In [ ]:
def test_img(img):
    data = []
    image = Image.open(img)
    image = image.resize((32,32))
    data.append(np.array(image))
    X_test = np.array(data)
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    return image, y_pred

In [ ]:
plot,prediction = test_img('Test/00010.png')
s = [str(i) for i in prediction] 
a = int("".join(s)) 
print("Predicted traffic sign is: ", classes[a])
plt.imshow(plot)
plt.show()